In [22]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import yfinance as yf
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np

In [20]:
df = pd.read_parquet("700077347251945472_label.parquet")

In [6]:
df = df.drop("author", axis=1)

In [115]:
df = df.drop("attachments", axis=1)

In [116]:
df = df.drop("embeds", axis=1)
df = df.drop("stickers", axis=1)
df = df.drop("reactions", axis=1)
df = df.drop("mentions", axis=1)
df = df.drop("ticker_len", axis=1)

In [7]:
df["stocktwits_score"] = df.stocktwits.apply(lambda x: 0 if x is None else (x[0]['score'] if x[0]['label'] == 'LABEL_1' else -x[0]['score']))

In [8]:
df["sentiment_score"] = df.pipe_sentiment_analysis.apply(lambda x: 0 if x is None else (x[0]['score'] if x[0]['label'] == 'POSITIVE' else -x[0]['score']))

In [120]:
df = df.drop("stocktwits", axis=1)
df = df.drop("pipe_sentiment_analysis", axis=1)

In [36]:
print(df.loc[492461].iloc[0].reactions)

[{'count': 6, 'emoji': {'id': '', 'imageUrl': 'https://twemoji.maxcdn.com/v/latest/svg/1f440.svg', 'isAnimated': False, 'name': '👀'}}]


In [30]:
def process_reaction(x):
    if len(x) == 0:
        return
    return [x[0]['emoji']
df['reactions'].apply(process_reaction)

7                                                      None
10                                                     None
10                                                     None
22                                                     None
167                                                    None
                                ...                        
492460                                                 None
492460                                                 None
492461    {'id': '', 'imageUrl': 'https://twemoji.maxcdn...
492461    {'id': '', 'imageUrl': 'https://twemoji.maxcdn...
492462    {'id': '', 'imageUrl': 'https://twemoji.maxcdn...
Name: reactions, Length: 227233, dtype: object

In [141]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(list(df.content))


Downloading: 100% 968/968 [00:00<00:00, 619kB/s]
Downloading: 100% 190/190 [00:00<00:00, 73.1kB/s]
Downloading: 100% 3.79k/3.79k [00:00<00:00, 2.08MB/s]
Downloading: 100% 645/645 [00:00<00:00, 281kB/s]
Downloading: 100% 122/122 [00:00<00:00, 97.2kB/s]
Downloading: 100% 471M/471M [00:09<00:00, 50.6MB/s] 
Downloading: 100% 53.0/53.0 [00:00<00:00, 41.6kB/s]
Downloading: 100% 5.07M/5.07M [00:00<00:00, 44.1MB/s]
Downloading: 100% 239/239 [00:00<00:00, 146kB/s]
Downloading: 100% 9.08M/9.08M [00:00<00:00, 47.1MB/s]
Downloading: 100% 480/480 [00:00<00:00, 273kB/s]
Downloading: 100% 14.8M/14.8M [00:00<00:00, 49.0MB/s]
Downloading: 100% 229/229 [00:00<00:00, 140kB/s]


In [163]:
df['embeddings'] = embeddings.tolist()

In [55]:
df.to_parquet("700077347251945472_embs.parquet")

In [56]:
final = df[df['diretion_today'].notna()][["stocktwits_score", "sentiment_score", "diretion_today", "content", "timestamp"]]

KeyError: "['stocktwits_score', 'sentiment_score'] not in index"

In [ ]:
train_data = TabularDataset(final[:int(len(final) * 0.7)])
test_data = TabularDataset(final[int(len(final) * 0.7):])

In [58]:
test_data.to_parquet("test_data.parquet")

In [ ]:
predictor = TabularPredictor(label='diretion_today').fit(train_data=train_data, num_gpus=8)

No path specified. Models will be saved in: "AutogluonModels/ag-20221124_080113/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221124_080113/"
AutoGluon Version:  0.6.0
Python Version:     3.7.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #53-Ubuntu SMP Wed Sep 18 13:35:53 UTC 2019
Train Data Rows:    153596
Train Data Columns: 4
Label Column: diretion_today
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [False, True]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression']

[1000]	valid_set's binary_error: 0.267093
[2000]	valid_set's binary_error: 0.246701
[3000]	valid_set's binary_error: 0.242703


	0.7613	 = Validation score   (accuracy)
	73.82s	 = Training   runtime
	0.5s	 = Validation runtime
Fitting model: LightGBM ...
	Training LightGBM with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1


[1000]	valid_set's binary_error: 0.252299
[2000]	valid_set's binary_error: 0.231108
[3000]	valid_set's binary_error: 0.227909
[4000]	valid_set's binary_error: 0.220312
[5000]	valid_set's binary_error: 0.219112
[6000]	valid_set's binary_error: 0.216713
[7000]	valid_set's binary_error: 0.218713


	0.7853	 = Validation score   (accuracy)
	107.54s	 = Training   runtime
	0.56s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7073	 = Validation score   (accuracy)
	166.59s	 = Training   runtime
	0.33s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7177	 = Validation score   (accuracy)
	181.27s	 = Training   runtime
	0.34s	 = Validation runtime
Fitting model: CatBoost ...
	Training CatBoost with GPU, note that this may negatively impact model quality compared to CPU training.


In [16]:
predictor = TabularPredictor.load("AutogluonModels/ag-20221124_185158/")

In [47]:
y_test = test_data["diretion_today"]
y_pred = predictor.predict(test_data.drop(columns=["diretion_today"]))
y_prob = predictor.predict_proba(test_data.drop(columns=["diretion_today"]))

In [50]:
test_data["y_pred"] = y_pred

In [51]:
test_data[["content", "y_pred"]]

,content,y_pred
376019,```fix\nTop Trending: CEMI SOFI LEXX CCIV SQ N...,True
376019,```fix\nTop Trending: CEMI SOFI LEXX CCIV SQ N...,True
376021,$NURO 5m,True
376028,CEMI curling off the lower trendline (again),True
376032,BTC ripping,True
...,...,...
492460,```fix\nTop Trending: LCID TOP ILAG TSLA NIO X...,True
492460,```fix\nTop Trending: LCID TOP ILAG TSLA NIO X...,True
492460,```fix\nTop Trending: LCID TOP ILAG TSLA NIO X...,True
492460,```fix\nTop Trending: LCID TOP ILAG TSLA NIO X...,True


In [18]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [19]:
perf

{'accuracy': 0.5688764659415446,
 'balanced_accuracy': 0.5165564955032926,
 'mcc': 0.05206957310818076,
 'f1': 0.7039061847925883,
 'precision': 0.5769158414994955,
 'recall': 0.9025819397993311}

In [ ]:
import autokeras as ak

clf = ak.ImageClassifier()
clf.fit(x_train, y_train)
results = clf.predict(x_test)

In [ ]:
final = df[df['diretion_today'].notna()][["stocktwits_score", "sentiment_score", "diretion_today"]]